## 3.2 The Gaussian Elimination Method and the LU Decomposition

**Implementation 3.1: Backward substitution**

We implement the backward insertion algorithm using `numpy`. We have to be careful with the indices. On paper, the indices of the entries of an $n$x$n$ matrix are usually numbered from $1$ to $n$, but recall that in most programming languages, including Python, the indices run from $0$ to $n-1$.

In [ ]:
import numpy as np

def backward(U, b):
    assert (len(b.shape) == 1), 'rhs is not a vector'
    n = b.shape[0]
    assert (U.shape == (n, n)), "Matrix dimensions don't match"
    
    x = np.empty_like(b)
    
    x[n - 1] = b[n - 1] / U[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        xr = 0
        for j in range(i + 1, n):
            xr += U[i, j] * x[j]
        x[i] = (b[i] - xr) / U[i, i]
    return x

*Additional code details*
- At the beginning of the function, we test whether the provided data is compatible, i.e. whether `b` is a vector and whether the matrix has the correct dimensions.
- The `numpy` function `np.empty_like()` creates a new, empty `array` with the same properties as `b`. This means that the dimensions and the precision with which floating point numbers are stored are the same.

We test our code using the following example
$$
U = \begin{pmatrix} 1 & 1 & 1 \\ 0 & 1 & 2 \\ 0 & 0 & 4 \end{pmatrix}
\quad\text{and}\quad
b = \begin{pmatrix} 1 \\ 1 \\ -4\end{pmatrix}.
$$

In [ ]:
U = np.array([[1, 1, 1], [0, 1, 2], [0, 0, 4]], dtype=np.float64)
b = np.array([1, 1, -4], dtype=np.float64)

Here we have specified that the matrix entries should be stored as 64-bit floating point numbers (otherwise known as `double`). This is important as `numpy` would otherwise infer the number format from the entries, which in this case would be `int64`, i.e. integers.

In [ ]:
x = backward(U, b)
print(' x = ', x)
print('Ux = ', np.dot(U, x))

Experiment with the entries of the matrix and the number format. For example, take the right-hand side $(1,1,-1)$ and save `U, b` in the data format `np.int64`. What do you observe, and can you explain your observation?

**Implementation 3.2: LU Factorisation without additional storage**

In [ ]:
def lu_simple(A):
    assert (A.shape[0] == A.shape[1]), 'The matrix is not square'
    n = A.shape[0]
    
    for i in range(0, n):
        for k in range(i + 1, n):
            A[k, i] = A[k, i] / A[i, i]
            for j in range(i + 1, n):
                A[k, j] = A[k, j] - A[k, i] * A[i, j]
    return None

Note that our function returns the type `None` to emphasise that we have modified the matrix *in-place*.


#### Example 3.8 (LU-Decomposition without pivoting)

Let us consider the example
$$
A = \begin{pmatrix}
2.3 & 1.8 & 1 \\ 1.4 & 1.1 & -0.7 \\ 0.8 & 4.3 & 2.1
\end{pmatrix}
\quad\text{and}\quad
b = \begin{pmatrix} 1.2 \\ -2.1 \\ 0.6 \end{pmatrix}.
$$

In [ ]:
A = np.array([[2.3, 1.8, 1], [1.4, 1.1, -0.7], [0.8, 4.3, 2.1]])
b = np.array([1.2, -2.1, 0.6])

It is not necessary to specify the number format here, as the arrays are filled directly with floating point numbers, which `numpy` interprets directly as `float64`. You can check this by displaying the data type using the `dtype` attribute of the arrays.

With the module `linalg` within `numpy` gives us an implementation to solve for $x$ directly:

In [ ]:
x_np = np.linalg.solve(A, b)
print('x_np = ', x_np)

We now create the matrix again with *half precision* floating point numbers to illustrate the effect of pivoting

In [ ]:
A = np.array([[2.3, 1.8, 1], [1.4, 1.1, -0.7], [0.8, 4.3, 2.1]], dtype=np.half)
b = np.array([1.2, -2.1, 0.6], dtype=np.half)

lu_simple(A)

print('Modified A =\n', A)

To solve the system, we also have to implement the forward substitution step. Here we assume that the diagonal elements of $L$ are normalised to be $1$.

**Implementation 3.3: Forward substitution**

In [ ]:
def forward(L, b):
    # We assume L has values 1 on the diagonal
    x = np.zeros_like(b)

    for i in range(0, b.shape[0]):
        xr = 0
        for j in range(0, i):
            xr += L[i, j] * x[j]
        x[i] = (b[i] - xr)
    return x

In [ ]:
y = forward(A, b)
print('y = ', y)
x = backward(A, y)
print('x = ', x)

Consequently, we have the relative error

In [ ]:
print('Relative error:', np.linalg.norm(x - x_np) / np.linalg.norm(x_np))

#### Example 3.9 (LU decomposition with pivoting)

We consider the same example, but this time including the pivot search 

**Implementation 3.4: LU decomposition with pivoting**

In [ ]:
def lu_pivot(A):
    assert (A.shape[0] == A.shape[1]), 'Matrix is not quadratic'
    n = A.shape[0]
    pivot = []

    for i in range(0, n):
        # Search for the pivot element and swap rows
        k = i
        for j in range(i, n):
            if abs(A[j, i]) > abs(A[k, i]):
                k = j
        A[[i, k], :] = A[[k, i], :]
        pivot.append([i, k])

        for k in range(i + 1, n):
            A[k, i] = A[k, i] / A[i, i]
            for j in range(i + 1, n):
                A[k, j] = A[k, j] - A[k, i] * A[i, j]

    return pivot

We apply this to the linear system:

In [ ]:
A = np.array([[2.3, 1.8, 1], [1.4, 1.1, -0.7], [0.8, 4.3, 2.1]], dtype=np.half)
b = np.array([1.2, -2.1, 0.6], dtype=np.half)

pivot = lu_pivot(A)
print('Modifed A =\n', A)

In [ ]:
for p in pivot:
    b[p] = b[[p[1], p[0]]] 
print('P b = ', b)

In [ ]:
y = forward(A, b)
print('y = ', y)
x = backward(A, y)
print('x = ', x)

The resulting relative error is

In [ ]:
np.linalg.norm(x - x_np) / np.linalg.norm(x_np)

Change the floating point precision to `np.single` or `np.double`. What do you observe? 